<a href="https://colab.research.google.com/github/constantinpape/training-deep-learning-models-for-vison/blob/master/day1/2_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression on CIFAR10

Now that we have seen how to load image data for classification and wrap it into a torch dataset, we will go ahead and train "simple" classiification models useing torch.
In this notebook, we will start with [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression)
that learns to classifiy the images based on the pixel values and then continue with a model trained on "hard-coded" convolutional features.

In [1]:
# import torch and other libraries
import os
import numpy as np
import torch
import torch.nn as nn

In [2]:
# check if we have gpu support.
# colab offers free gpus, however they are not activated by default.
# to activate the gpu, go to 'Runtime->Change runtime type'. 
# Then select 'GPU' in 'Hardware accelerator' and click 'Save'
print(torch.cuda.is_available())

# we need to define the device for torch, yadda yadda
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

False


In [3]:
# we collect some functions we need from previous notebooks that we 
# want to reuse in 'utils.py'
# TODO need to get this into colab
import utils

In [4]:
# we assume that you have downloaded the
# cifar dataset already, if not, rerun this:
# !cifar2png cifar10 cifar10

In [5]:
# RUN THIS IF YOU USE THIS NOTEBOOK IN GOOGLE COLAB
# TODO

In [6]:
# RUN THIS IF YOU USE THIS NOTEBOOK LOCALLY

cifar_dir = './cifar10'

## Prepare the data

We have already seen how to create a torch dataset for the cifar dataset in the previous notebook. We will repeat this here, but now split our data into training and validation data beforehand  

In [7]:
images, labels = utils.load_cifar(os.path.join(cifar_dir, 'train'))

100%|██████████| 10/10 [00:18<00:00,  1.85s/it]


In [8]:
# shuffle the data, split into training and validation set

# shuffle the data
n_images = len(images)
indices = np.arange(n_images)
np.random.shuffle(indices)

# split into training and validation data
validation_fraction = 0.15
split_index = int(validation_fraction * n_images)
train_indices = indices[:-split_index]
val_indices = indices[-split_index:]

train_images, val_images = images[train_indices], images[val_indices]
train_labels, val_labels = labels[train_indices], labels[val_indices]
assert len(train_images) == len(train_labels)
assert len(val_images) == len(val_labels)
assert len(train_images) + len(val_images) == n_images

print("Split cifar into training and validation data:")
print("Have", len(train_images), "training images and", len(val_images), "validation images")

Split cifar into training and validation data:
Have 42500 training images and 7500 validation images


In [9]:
# create the torch datasets for training and validation
from functools import partial

# TODO what is actually the clean way here? determine all on train?
mean, std = np.mean(train_images), np.std(train_images)
trafos = [
    utils.to_channel_first,
    partial(utils.normalize, mean=mean, std=std),
    utils.to_tensor
]
trafos = partial(utils.compose, transforms=trafos)

train_dataset = utils.DatasetWithTransform(train_images, train_labels,
                                           transform=trafos)

val_dataset = utils.DatasetWithTransform(val_images, val_labels,
                                         transform=trafos)

## Training and validation function

Now write the function that runs training and validation for 1 epoch.
Epoch: training model on the training set once.
Important: we want to monitor the progress during the training -> tensorboard.

In [31]:
# this is for tensorboard
# from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard import SummaryWriter

# this is the training function.
# don't be scared by the many arguments, a lot of this is just
# for logging, and we explain it in more detail in the
# training procedure
def train(model, loader, 
          loss_function, optimizer, device, epoch,
          log_frequency=10, tb_logger=None,
          log_image_interval=20):
    """ Train model for one epoch.
    
    Arguments:
        model - 
        loader - 
        loss_function - 
        optimizer - 
        device - 
        epoch - 
        log_frequency - 
        tb_logger - 
        log_image_interval -
    """

    # set model to train mode
    model.train()
    
    # iterate over the training batches provided by the loader
    n_batches = len(loader)
    log_interval_ = n_batches // log_frequency
    for batch_id, (x, y) in enumerate(loader):
       
        # send tensors to the active device
        x.to(device), y.to(device)
        
        # set the gradients to zero, to start with "clean" gradients
        # in this training iteration
        optimizer.zero_grad()
        
        # apply the model and get our prediction
        prediction = model(x)
        
        # FIXME collate in dataloader is annoying
        # calculate the loss (negative log likelihood loss)
        loss_value = loss_function(prediction, y[:, 0])
        
        # calculate the gradients (`loss.backward()`) 
        # and apply them to the model weights (`optimizer.ste()`)
        loss_value.backward()
        optimizer.step()
        
        # report the training progress
        if batch_id % log_interval_ == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_id * len(x), n_batches,
                  100. * batch_id / n_batches, loss_value.item()))
            
        if tb_logger is not None:
            step = epoch * n_batches + batch_id
            tb_logger.add_scalar(tag='train-loss', 
                                 scalar_value=loss_value.item(),
                                 global_step=step)
            # TODO add the prediction and label here
            if log_image_interval is not None and step % log_image_interval == 0:
                tb_logger.add_images(tag='input', 
                                     img_tensor=x.to('cpu'),
                                     global_step=step)

In [22]:
# yadda yadda
def validate(model, loader, loss_function, device, 
             epoch, step, tb_logger=None):
    n_batches = len(loader)
    n_samples = len(loader.dataset)
   
    correct = 0
    mean_loss = 0
    
    with torch.no_grad():
        for x, y in loader:
            x.to(device), y.to(device)
            prediction = model(x)
            
            # FIXME collate in dataloader is annoying
            mean_loss += loss_function(prediction, y[:, 0]).item()
 
            # compute the most likely class predictions
            prediction = prediction.max(1, keepdim=True)[1]
            correct += prediction.eq(y.view_as(prediction)).sum().item()
    
    accuracy = correct / n_samples
    mean_loss /= n_batches
    print("Validate Epoch: {} Loss: {:.6f}, Accuracy: {:.6f}".format(
        epoch, mean_loss, accuracy
    ))
    if tb_logger is not None:
        tb_logger.add_scalar(tag="validation-accuracy",
                             global_step=step,
                             scalar_value=accuracy)
        tb_logger.add_scalar(tag="validation-loss",
                             global_step=step,
                             scalar_value=mean_loss)

## Model

different ways to define models, inherit from `torch.nn.Module`,
define forward pass via method `forward`, yadda yadda

In [12]:
# define logistic regression model
class LogisticRegressor(nn.Module):
    def __init__(self, n_pixels, n_classes):
        super().__init__()
        self.n_pixels = n_pixels
        self.n_classes = n_classes
        self.log_reg = nn.Sequential(nn.Linear(self.n_pixels,
                                               self.n_classes),
                                     nn.LogSoftmax(dim=1))
        
    def forward(self, x):
        # reshape the input to be 1d instead of 2d,
        # which is required for fully connected layers
        x = x.view(-1, self.n_pixels)
        x = self.log_reg(x)
        return x

## Set up the training

Now it's time to bring everything together:

In [13]:
n_pixels = 3 * 32 * 32  # number channels * number pixels
n_classes = 10    
model = LogisticRegressor(n_pixels, n_classes)
model.to(device)

LogisticRegressor(
  (log_reg): Sequential(
    (0): Linear(in_features=3072, out_features=10, bias=True)
    (1): LogSoftmax(dim=1)
  )
)

In [14]:
# create data loader
# what is this yadda yadda
from torch.utils.data import DataLoader
batch_size = 4
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size, 
                          shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=10)

In [15]:
# create optimizer
# what is this yadda yadda
from torch.optim import Adam
optimizer = Adam(model.parameters())

In [16]:
# create loss
# what is this yadda yadda
loss_function = nn.NLLLoss()
loss_function.to(device)

NLLLoss()

In [17]:
# TODO explain how to access the tensorboard
tb_logger = SummaryWriter('runs/log_reg')
%tensorboard --logdir runs

UsageError: Line magic function `%tensorboard` not found.


In [23]:
n_epochs = 4
for epoch in range(n_epochs):
    train(model, train_loader, loss_function, optimizer,
          device, epoch, tb_logger=tb_logger)
    step = (epoch + 1) * len(train_loader)
    validate(model, val_loader, loss_function, device, epoch, step,
             tb_logger=tb_logger)

Train Epoch: 0 [0/10625 (0%)]	Loss: 193.064514
Train Epoch: 0 [4248/10625 (10%)]	Loss: 202.434006
Train Epoch: 0 [8496/10625 (20%)]	Loss: 471.633423
Train Epoch: 0 [12744/10625 (30%)]	Loss: 224.412338
Train Epoch: 0 [16992/10625 (40%)]	Loss: 111.017883
Train Epoch: 0 [21240/10625 (50%)]	Loss: 276.172302
Train Epoch: 0 [25488/10625 (60%)]	Loss: 570.844482
Train Epoch: 0 [29736/10625 (70%)]	Loss: 343.677063
Train Epoch: 0 [33984/10625 (80%)]	Loss: 173.348755
Train Epoch: 0 [38232/10625 (90%)]	Loss: 361.159454
Train Epoch: 0 [42480/10625 (100%)]	Loss: 283.078308
Validate Epoch: 0 Loss: 306.538346, Accuracy: 0.240667
Train Epoch: 1 [0/10625 (0%)]	Loss: 275.397980
Train Epoch: 1 [4248/10625 (10%)]	Loss: 241.823090
Train Epoch: 1 [8496/10625 (20%)]	Loss: 293.156616
Train Epoch: 1 [12744/10625 (30%)]	Loss: 247.963928
Train Epoch: 1 [16992/10625 (40%)]	Loss: 650.058350
Train Epoch: 1 [21240/10625 (50%)]	Loss: 195.868988
Train Epoch: 1 [25488/10625 (60%)]	Loss: 100.993896
Train Epoch: 1 [29736/

## Check the model on test data

TODO need to check everything on test data!
confusion matrix

In [29]:
# evaluate the model on the validation data
val_images = torch.from_numpy(val_images)
val_labels = torch.from_numpy(val_labels)
val_dataset = TensorDataset(val_images, val_labels)

In [30]:
val_loader = DataLoader(val_dataset, batch_size=25)

In [34]:
def validate(model, loader):
    correct = 0
    with torch.no_grad():
        for x, y in loader:
            prediction = model(x)
            # compute the most likely class predictions
            prediction = prediction.max(1, keepdim=True)[1]
            correct += prediction.eq(y.view_as(prediction)).sum().item()
    accuracy = correct / len(loader.dataset)
    return accuracy

In [35]:
model_accuracy = validate(model, val_loader)
print(model_accuracy)

0.3586666666666667


## Logistic Regression with preset filters

In [26]:
# write this as a function that takes 
def apply_filters(image, target, filter_list, keep_image=False):    
    filtered = [image] if keep_image else [] 
    for filter_function in filter_list:
        filtered.append(filter_function(image))
    data = np.concatenate(filtered, axis=-1)
    return data, target

In [27]:
import skimage.filters as filters

# simple filter training
filter_list = [filters.gaussian]
blur_filter = partial(apply_filters,
                      filter_list=filter_list,
                      keep_image=True)


trafos = [
    blur_filter,
    partial(utils.normalize, mean=mean, std=std),
    utils.to_channel_first,
    utils.to_tensor
]
trafos = partial(utils.compose, transforms=trafos)

train_dataset = utils.DatasetWithTransform(train_images, train_labels,
                                           transform=trafos)

val_dataset = utils.DatasetWithTransform(val_images, val_labels,
                                         transform=trafos)


In [29]:
batch_size = 4
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size, 
                          shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=10)

In [32]:
n_pixels = 6 * 32 * 32  # number channels * number pixels
n_classes = 10    
model = LogisticRegressor(n_pixels, n_classes)
model.to(device)

optimizer = Adam(model.parameters())

tb_logger = SummaryWriter('runs/log_reg')

n_epochs = 4
for epoch in range(n_epochs):
    train(model, train_loader, loss_function, optimizer,
          device, epoch, tb_logger=tb_logger, log_image_interval=None)
    step = (epoch + 1) * len(train_loader)
    validate(model, val_loader, loss_function, device, epoch, step,
             tb_logger=tb_logger)

<ipython-input-26-94a7f6c16c16>:5: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  filtered.append(filter_function(image))


Train Epoch: 0 [0/10625 (0%)]	Loss: 46.520142
Train Epoch: 0 [4248/10625 (10%)]	Loss: 443.046082
Train Epoch: 0 [8496/10625 (20%)]	Loss: 100.968681
Train Epoch: 0 [12744/10625 (30%)]	Loss: 277.865112
Train Epoch: 0 [16992/10625 (40%)]	Loss: 297.760437
Train Epoch: 0 [21240/10625 (50%)]	Loss: 676.625366
Train Epoch: 0 [25488/10625 (60%)]	Loss: 332.831421
Train Epoch: 0 [29736/10625 (70%)]	Loss: 324.362518


KeyboardInterrupt: 

## Tasks and Questions:

Questions:
- What accuracy do the different models reach?
- Which accuracy do you expect by guessing?
- Other metrics than accuracy / confusion table?

Tasks:
- Try more filters, compare the different models via tensorboard and on the test dataset.
- Filters can be expressed as torch convolutions! Do this for torch.gaussian and train a model with this for different degrees of smoothing. How does it perform? (Give a hint!)
- Advanced can you do it for other convolutional filters?